<a href="https://colab.research.google.com/github/wyllton/phyton/blob/master/Desafio_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import pandas as pd
from google.colab.patches import cv2_imshow
import os
import zipfile
import tensorflow
import keras
from keras.layers import Conv2D, Activation, MaxPooling2D, Dense, Flatten, Dropout,BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import model_from_json


In [2]:
path = '/content/drive/My Drive/archive.zip'
zip = zipfile.ZipFile(file = path, mode = 'r')
zip.extractall('./')

In [3]:
categorias = {'01_palm' : 0 , '02_l' : 1 , '03_fist' : 2 , '04_fist_moved': 3 , '05_thumb': 4, '06_index' : 5
              , '07_ok': 6, '08_palm_moved': 7 , '09_c': 8 , '10_down': 9}
img_size = 150

In [4]:
x_dados=[]
y_dados = []
num_fotos = 0
path = "/content/leapgestrecog/leapGestRecog/0"
for i in range (0, len(categorias)):
  for j in os.listdir(path + str(i) + '/' ):
    if not j.startswith('.'):
      count = 0
      for k in os.listdir(path + str(i) + '/' + j + '/'):
        img = cv2.imread(path + str(i) + '/' + j +'/' + k)
        img = cv2.resize(img, (img_size, img_size))
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        arr = np.array(img)
        x_dados.append(arr)
        count = count + 1
      y_valor = np.full((count, 1), categorias[j])
      y_dados.append(y_valor)
      num_fotos = num_fotos + count
x_dados = np.array(x_dados, dtype='float32')
x_dados = x_dados / 255
y_dados = np.array(y_dados)
y_dados = y_dados.reshape(num_fotos, 1)
y_dados = keras.utils.to_categorical(y_dados)

      

In [5]:
x_dados = x_dados.reshape((num_fotos, img_size, img_size, 1))
x_dados.shape

(20000, 150, 150, 1)

In [6]:
x_treino, x_teste , y_treino , y_teste = train_test_split(x_dados, y_dados, test_size = 0.2, random_state = 45)
_, x_val, _ , y_val = train_test_split(x_treino, y_treino , test_size = 0.2, random_state = 43)

In [7]:
num_features = 32
num_labels = 10
batch_size = 64
epochs = 10

modelo = Sequential()

modelo.add(Conv2D(num_features, kernel_size= (5,5), activation= 'relu', 
                 input_shape =(img_size, img_size , 1), kernel_regularizer = l2(0.01),padding='same'))
modelo.add(MaxPooling2D(pool_size = (2,2), strides=(2,2)))

modelo.add(Conv2D(2*num_features, kernel_size = (3,3), activation= 'relu', padding='same'))
modelo.add(MaxPooling2D(pool_size = (2,2), strides=(2,2)))

modelo.add(Conv2D(2*2*num_features, kernel_size = (3,3), activation= 'relu', padding='same'))
modelo.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))



modelo.add(Flatten())

modelo.add(Dense(2*2*num_features, activation= 'relu'))
modelo.add(Dense(num_labels, activation= 'softmax'))

modelo.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 32)      832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 41472)             0

In [8]:
modelo.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr = 0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics = ['accuracy'])

arquivo_modelo = "modelo_01_DesafioIA.h5"
arquivo_modelo_json = "modelo_01_DesafioIa.json"

lr_reducer = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.9 , patience=3, verbose= 1)

early_stopper = EarlyStopping(monitor='val_loss', min_delta= 0, patience = 4, verbose=1 )

checkpointer = ModelCheckpoint(arquivo_modelo, monitor = 'val_loss', verbose = 1, 
                               save_best_only= True)



In [9]:
modelo_json = modelo.to_json()
with open(arquivo_modelo_json, 'w') as json_file:
  json_file.write(modelo_json)


In [10]:
history = modelo.fit(x_treino,y_treino,
                     batch_size = batch_size,
                     epochs = epochs, verbose = 1,
                     validation_data = (x_val, y_val),shuffle = True,
                     callbacks = [lr_reducer, early_stopper, checkpointer])

Epoch 1/10
250/250 [==============================] - ETA: 0s - loss: 0.2511 - accuracy: 0.9243
Epoch 00001: val_loss improved from inf to 0.02310, saving model to modelo_01_expressões.h5
250/250 [==============================] - 626s 3s/step - loss: 0.2511 - accuracy: 0.9243 - val_loss: 0.0231 - val_accuracy: 0.9975
Epoch 2/10
250/250 [==============================] - ETA: 0s - loss: 0.0229 - accuracy: 0.9975
Epoch 00002: val_loss improved from 0.02310 to 0.01399, saving model to modelo_01_expressões.h5
250/250 [==============================] - 627s 3s/step - loss: 0.0229 - accuracy: 0.9975 - val_loss: 0.0140 - val_accuracy: 0.9994
Epoch 3/10
250/250 [==============================] - ETA: 0s - loss: 0.0115 - accuracy: 0.9998
Epoch 00003: val_loss improved from 0.01399 to 0.00899, saving model to modelo_01_expressões.h5
250/250 [==============================] - 628s 3s/step - loss: 0.0115 - accuracy: 0.9998 - val_loss: 0.0090 - val_accuracy: 0.9997
Epoch 4/10
250/250 [============

In [11]:
modelo.save('modelo_01_DesafioIA.h5')

In [12]:
tLoss, tAccuracy = modelo.evaluate(x_teste, y_teste)
print('Teste Accuracy: {:2.2f}%'.format(tAccuracy*100))

125/125 [==============================] - 41s 331ms/step - loss: 0.0020 - accuracy: 1.0000
Teste Accuracy: 100.00%
